# Image Filtering: Spatial Methods

In [ ]:
%load_ext cython

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage.util import random_noise
from skimage.util import img_as_float32 as img_as_float

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
def show_plots(I, titles=None):
    fig, ax = plt.subplots(1, len(I), figsize=(12,2))
    
    r = I[0].shape[0]//2
    
    for i in np.arange(0,len(I)):
        ax[i].plot(I[i][r,:])
        ax[i].set_ylim(0,1)
        ax[i].set_axis_off()
        if titles != None:
            ax[i].set_title(titles[i])

In [ ]:
def show_images(I, titles=None):
    fig, ax = plt.subplots(1, len(I), figsize=(12,12))
    
    for i in np.arange(0,len(I)):
        ax[i].imshow(I[i], cmap='gray')
        ax[i].set_axis_off()
        if titles != None:
            ax[i].set_title(titles[i])

In [ ]:
I1 = io.imread("../../images/umbc.png", as_gray=True)
I1 = img_as_float(I1)

noise_std = 0.05
In = random_noise(I1, var=noise_std**2)
In = img_as_float(In)

show_plots([I1, In], ['Original','Noisy Image'])
show_images([I1, In], ['Original','Noisy Image'])

## Non-Linear Filter: Anisotropic Diffusion

In [ ]:
%%cython 
import numpy as np

cpdef g1(float v, float K):
    return np.exp(-(v/K)**2)

cpdef g2(float v, float K):
    return 1/(1+(v/K)**2)

cpdef diffusion_filter(I0, float alpha=0.10, int mode=1, float K=1.0):
    g = [g1, g2]
    I1 = np.empty_like(I0)
    
    cdef float CHG
    cdef float Iuv, Imn
    cdef int u, v, m, n
    
    Iz = np.pad(I0, ((1,1),(1,1)), mode='wrap')
    N4 = np.array([[0,-1],[1,0],[0,1],[-1,0]])
    
    for u in np.arange(0,I0.shape[0]):
        for v in np.arange(0,I0.shape[1]):            
            Iuv = I0[u][v]
            
            CHG = 0.0
            for m, n in N4:
                Imn = Iz[u+m+1][v+n+1]
                CHG += g[mode-1](Imn-Iuv, K)*(Imn-Iuv)

            I1[u][v] = Iuv + alpha*CHG
        
    return I1

In [ ]:
%%time
alpha = 0.25
K = noise_std

I2 = [ In ]
for k in np.arange(0,5):
    Ip = diffusion_filter(I2[-1], alpha=alpha, mode=1, K=K)
    I2.append(Ip)

In [ ]:
show_plots([I2[1],I2[-1]], ['Iteration 1','Iteration N'])
show_images([I2[1],I2[-1]],['Iteration 1','Iteration N'])

In [ ]:
%%time
alpha = 0.25
K = noise_std

I2 = [ In ]
for k in np.arange(0,5):
    Ip = diffusion_filter(I2[-1], alpha=alpha, mode=2, K=K)
    I2.append(Ip)

In [ ]:
show_plots([I2[1],I2[-1]], ['Iteration 1','Iteration N'])
show_images([I2[1],I2[-1]],['Iteration 1','Iteration N'])